In [55]:
import pickle
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier

In [56]:
if os.path.exists("bitcoin_trades.csv"):
    bitcoinData = pd.read_csv("bitcoin_trades.csv", index_col=0)
    bitcoinData.index = pd.to_datetime(bitcoinData.index, unit='s')
    
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2011-12-31 07:52:00,4.39,4.39,4.39,4.39,0.455581,2.000000,4.390000
2011-12-31 07:53:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:54:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:55:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-31 07:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496


In [57]:
bitcoinData = bitcoinData.loc["2019-01-01":]
bitcoinData = bitcoinData.dropna()
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price
Timestamp,,,,,,,
2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,20301.156505,3696.974936
2019-01-01 00:01:00,3694.72,3694.72,3690.65,3690.65,9.500151,35080.265871,3692.600865
2019-01-01 00:02:00,3689.73,3689.73,3686.62,3686.62,0.965966,3562.371230,3687.884698
2019-01-01 00:03:00,3692.85,3692.85,3688.32,3692.35,0.296662,1095.220713,3691.813285
2019-01-01 00:04:00,3692.35,3692.35,3690.34,3690.34,0.111622,412.065433,3691.614849
...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496


In [58]:
bitcoinData["next_minute"] = bitcoinData["Close"].shift(-1)

bitcoinData['Up'] = (bitcoinData["Close"] < bitcoinData["next_minute"]).astype(int)
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,next_minute,Up
Timestamp,,,,,,,,,
2019-01-01 00:00:00,3693.85,3698.79,3693.85,3698.00,5.491289,20301.156505,3696.974936,3690.65,0
2019-01-01 00:01:00,3694.72,3694.72,3690.65,3690.65,9.500151,35080.265871,3692.600865,3686.62,0
2019-01-01 00:02:00,3689.73,3689.73,3686.62,3686.62,0.965966,3562.371230,3687.884698,3692.35,1
2019-01-01 00:03:00,3692.85,3692.85,3688.32,3692.35,0.296662,1095.220713,3691.813285,3690.34,0
2019-01-01 00:04:00,3692.35,3692.35,3690.34,3690.34,0.111622,412.065433,3691.614849,3690.85,1
...,...,...,...,...,...,...,...,...,...
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58685.81,0
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58723.84,1
2021-03-30 23:58:00,58693.43,58723.84,58693.43,58723.84,1.705682,100117.070370,58696.198496,58760.59,1


In [59]:
horizons = [1,3,5,20,60]

predictors = []

for horizon in horizons:
    rolling_averages = bitcoinData.rolling(horizon).mean()
    
    ratio_column = f"Close_Ratio_{horizon}"
    bitcoinData[ratio_column] = bitcoinData["Close"] / rolling_averages["Close"]
    
    trend_column = f"Trend_{horizon}"
    
    bitcoinData[trend_column] = bitcoinData.shift(1).rolling(horizon).sum()["Up"]
    
    predictors += [ratio_column, trend_column]
    

In [60]:
bitcoinData = bitcoinData.dropna()
bitcoinData

,Open,High,Low,Close,Volume_(BTC),Volume_(Currency),Weighted_Price,next_minute,Up,Close_Ratio_1,Trend_1,Close_Ratio_3,Trend_3,Close_Ratio_5,Trend_5,Close_Ratio_20,Trend_20,Close_Ratio_60,Trend_60
Timestamp,,,,,,,,,,,,,,,,,,,
2019-01-01 01:20:00,3692.09,3693.35,3692.09,3692.09,0.213983,790.108310,3692.380805,3689.99,0,1.0,1.0,0.999974,1.0,0.999705,2.0,0.999789,11.0,0.999818,27.0
2019-01-01 01:22:00,3690.63,3690.63,3689.99,3689.99,0.011386,42.017298,3690.376431,3688.85,0,1.0,0.0,0.999688,1.0,0.999427,1.0,0.999256,11.0,0.999253,27.0
2019-01-01 01:24:00,3691.18,3691.94,3688.85,3688.85,9.637709,35563.719624,3690.059317,3691.85,1,1.0,0.0,0.999604,1.0,0.999396,1.0,0.999014,10.0,0.998934,27.0
2019-01-01 01:25:00,3691.49,3691.85,3691.49,3691.85,1.113608,4111.264790,3691.841141,3689.74,0,1.0,1.0,1.000439,1.0,1.000278,2.0,0.999861,10.0,0.999749,27.0
2019-01-01 01:26:00,3691.94,3691.94,3685.34,3689.74,2.221163,8187.804861,3686.269281,3691.65,1,1.0,0.0,0.999890,1.0,0.999793,2.0,0.999359,9.0,0.999180,27.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-30 23:55:00,58742.18,58742.18,58714.31,58714.31,2.519999,148004.448110,58731.946927,58686.00,0,1.0,0.0,0.999944,1.0,1.000100,3.0,1.001052,8.0,1.000764,31.0
2021-03-30 23:56:00,58714.31,58714.31,58686.00,58686.00,1.384487,81259.372187,58692.753339,58685.81,0,1.0,0.0,0.999533,1.0,0.999630,2.0,1.000556,8.0,1.000287,30.0
2021-03-30 23:57:00,58683.97,58693.43,58683.97,58685.81,7.294848,428158.146640,58693.226508,58723.84,1,1.0,0.0,0.999837,0.0,0.999675,1.0,1.000503,8.0,1.000291,29.0


In [82]:
testData = bitcoinData[-100000:]

def predict(train, test, predictors, model):
    model.fit(train[predictors], train["Up"])
    preds = model.predict_proba(test[predictors])[:,1]
    preds[preds >= .65] = 1
    preds[preds < .65] = 0
    preds = pd.Series(preds, index=test.index, name="Predictions")
    combined = pd.concat([test["Up"], preds], axis=1)
    return combined

def backtest(data, model, predictors, start=750000, step=250):
    all_predictions = []
    end = 15

    for i in range(start, data.shape[0], step):
        train = data.iloc[i-2500:i].copy()
        test = data.iloc[i:(i+step)].copy()
        predictions = predict(train, test, predictors, model)
        all_predictions.append(predictions)
        end -= 1
        print(i)
        if end < 1:
            break
        
    
    return pd.concat(all_predictions)

In [62]:
from sklearn.ensemble import RandomForestClassifier


model = RandomForestClassifier(n_estimators=250,min_samples_split=75, random_state=1)
train = testData[:-200]
test = testData[-200:]

model.fit(train[predictors], train["Up"])



RandomForestClassifier(min_samples_split=75, n_estimators=250, random_state=1)

In [76]:
from sklearn.metrics import precision_score

preds = model.predict(test[predictors])
preds = pd.Series(preds, index=test.index)
precision_score(test["Up"], preds)

0.5443037974683544

In [83]:
predictions = backtest(bitcoinData, model, predictors)


750000
750250
750500
750750
751000
751250
751500
751750
752000
752250
752500
752750
753000
753250
753500


In [85]:
predictions["Predictions"].value_counts()

Predictions
0.0    3125
1.0     625
Name: count, dtype: int64

In [84]:
precision_score(predictions["Up"], predictions["Predictions"])

0.6608

In [86]:
import pickle

with open("bittybotV1.pkl", 'wb') as f:
    pickle.dump(model,f)

In [87]:
predictors

['Close_Ratio_1',
 'Trend_1',
 'Close_Ratio_3',
 'Trend_3',
 'Close_Ratio_5',
 'Trend_5',
 'Close_Ratio_20',
 'Trend_20',
 'Close_Ratio_60',
 'Trend_60']